<a href="https://colab.research.google.com/github/CoelhoMestre/Projeto-Alura-Google-Gemini/blob/main/Projeto_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q -U google-generativeai

In [ ]:
# Instalar Framework ADK de agentes do Google
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [ ]:
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown, HTML
import requests
import warnings
import ipywidgets as widgets
from typing import List, Dict, Any
import re


warnings.filterwarnings("ignore")

In [ ]:
# Configuração da chave da API do Google
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
#==============================================#
#======= Agente 1: Buscador de Conteúdo =======#
#==============================================#
def agente_buscador(conteudo: str) -> str:
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especializado em gerar informações para quizzes educativos.
        Utilize a ferramenta google_search no idioma portugues para encontrar conteúdo relevante sobre a [disciplina] e o tópico [conteúdo],
        **Verifique se a disciplina tem relação com o conteúdo! Por exemplo, se o conteúdo for 2º guerra mundial de a disciplina for Artes,
        vocé terá que relacionar a disciplina com o conteúdo** garantindo que as informações sejam adequadas para o nível de ensino correspondente.
        Busque por uma visão geral do tema e identifique até [10] subtópicos ou conceitos chave importantes.
        Gere um parágrafo introdutório com no máximo [10] frases, apresentando o tema principal de forma clara e concisa
        e mencionando brevemente alguns dos subtópicos encontrados. Utilize uma linguagem acessível e destaque a relevância
        do conteúdo para o aprendizado.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )
    entrada = f"Conteúdo: {conteudo}"
    informacoes = call_agent(buscador, entrada)
    return informacoes

In [ ]:
#==============================================#
#======= Agente 2: Gerador de Perguntas =======#
#==============================================#
def agente_gerador_quiz(informacoes: str, disciplina: str, turma: str) -> str:
    """Gera as perguntas do quiz (retorna string para revisão)."""
    gerador = Agent(
        name="agente_gerador_quiz",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em elaboração de perguntas para quizzes educativos. Sua tarefa é analisar as informações
        fornecidas sobre um determinado conteúdo da disciplina [disciplina] para a turma [turma].
        Com base nessas informações, formule **apenas 10 possíveis perguntas, não pode ter mais** relevantes para o conteúdo
        e adequadas ao nível de ensino da turma. Atenção! as perguntas devem ser de marcar, não pode ser resposta escrita.
        Para cada pergunta, apresente a pergunta de forma clara e concisa. **Não inclua as opções de resposta ou a resposta
        correta nesta etapa.**
        Fomate a saída como uma lista numerada de perguntas, onde cada pergunta aparece em uma nova linha.
        """,
        description="Agente que gera perguntas de quiz",
    )
    entrada = f"Informações: {informacoes}\nDisciplina: {disciplina}\nTurma: {turma}"
    quiz = call_agent(gerador, entrada)
    return quiz

In [ ]:
#==============================================#
#======= Agente 3: Revisor de Perguntas =======#
#==============================================#
def agente_revisor_formatador(quiz: str) -> str:
    """Revisa a qualidade das perguntas e formata as questões"""

    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um revisor e seletor de perguntas para quizzes educativos.
        Sua tarefa é analisar a lista de perguntas fornecida, geradas com base no conteúdo [conteudo] para
        a disciplina de [disciplina] da turma [turma].
        Para cada pergunta, avalie se ela é clara, concisa, gramaticalmente correta e adequada ao nível de ensino da turma.
        Verifique se a pergunta é relevante para o conteúdo especificado. Após a análise, selecione as perguntas que você
        considera mais adequadas e de maior qualidade para formar um quiz eficaz para esta turma e conteúdo. As perguntas
        selecionadas devem cobrir os aspectos mais importantes do tema.
        A saída deve ser formatada para ser facilmente lido.
        Adicione numeração para cada pergunta (Questão X) e letras (A, B, C, D, E) para cada opção de resposta.
        """,
        description="Agente que revisa a qualidade do quiz"
    )
    entrada = f"Quiz: {quiz}"
    quiz_revisado_formatado = call_agent(revisor, entrada)
    return quiz_revisado_formatado

In [ ]:
#==============================================#
#======= Agente 4: Formatador de Quiz =========#
#==============================================#
def agente_solucionador(quiz_revisado: str) -> str:
    """Formata o quiz revisado para apresentação."""
    formatador = Agent(
        name="agente_formatador_quiz",
        model="gemini-2.0-flash",
        instruction="""
        Você é um solucionador de quiz.
        Você vai analisar o quiz elaborado e solucioná-lo, apresentando as opções corretas de cada questão anteriormente elaborada.
        """,
        description="Agente que formata o quiz revisado para apresentação"
    )
    entrada = f"Quiz Revisado: {quiz_revisado}"
    quiz_solucionado = call_agent(formatador, entrada)
    return quiz_solucionado

In [ ]:
#==============================================#
#======= Agente 5: Explicador de Quiz =========#
#==============================================#
def agente_gerador_explicacoes(quiz_revisado: str, quiz_solucionado: str, conteudo: str) -> str:
    """Gera explicações detalhadas para as respostas corretas e incorretas do quiz."""
    gerador_explicacoes = Agent(
        name="agente_gerador_explicacoes",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em fornecer explicações detalhadas para quizzes educacionais.
        Com base no quiz revisado: "{quiz_revisado}" e na sua solução: "{quiz_solucionado}",
        para o conteúdo de "{conteudo}", sua tarefa é gerar explicações claras e concisas para cada questão.

        Para cada questão, você deve:
        1. Indicar qual é a alternativa correta.
        2. Explicar o raciocínio por trás da resposta correta, detalhando o conceito ou princípio envolvido.
        3. Para cada uma das alternativas incorretas, explicar por que elas estão erradas, identificando o erro conceitual ou a informação incorreta que elas apresentam.
        4. Mantenha um tom didático e acessível, visando a compreensão do aluno.

        Formato de saída desejado:
        Questão 1:
        - Correta: [Letra da alternativa correta] - [Explicação da resposta correta]
        - Incorretas:
          - [Letra da alternativa incorreta]: [Explicação de por que está incorreta]
          - [Letra da alternativa incorreta]: [Explicação de por que está incorreta]
          - [Letra da alternativa incorreta]: [Explicação de por que está incorreta]

        Questão 2:
        - Correta: ...
        - Incorretas:
          - ...
          - ...
          - ...

        ... e assim por diante para todas as questões do quiz.
        """,
        description="Agente que gera explicações detalhadas para as respostas do quiz"
    )
    entrada = f"Quiz Revisado: {quiz_revisado}\nSolução do Quiz: {quiz_solucionado}\nConteúdo: {conteudo}"
    explicacoes_detalhadas = call_agent(gerador_explicacoes, entrada)
    return explicacoes_detalhadas

In [ ]:
#==============================================#
#======= Agente 6: Indicador de Fontes ========#
#==============================================#
def agente_indicador_fontes(quiz_original: str, conteudo_pesquisado: str) -> str:
    """Sugere as possíveis fontes de onde as questões do quiz podem ter sido elaboradas."""
    sugestor_fontes = Agent(
        name="agente_sugestor_fontes",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em sugerir fontes para a elaboração de quizzes educacionais.
        Com base no seguinte quiz original: "{quiz_original}" e no conteúdo pesquisado: "{conteudo_pesquisado}",
        sua tarefa é indicar as fontes prováveis que um professor ou criador de conteúdo utilizaria para elaborar essas questões.
        Liste as sugestões de fontes, como:
        - Livros didáticos relevantes para "{conteudo_pesquisado}" e a disciplina relacionada.
        - Sites educacionais confiáveis que abordam o tema.
        - Artigos acadêmicos ou de divulgação científica (se aplicável ao nível do quiz).
        - Provas anteriores de vestibulares, concursos ou avaliações similares.
        - Materiais de estudo e resumos sobre "{conteudo_pesquisado}".
        Seja específico ao sugerir tipos de materiais e possíveis palavras-chave para encontrar essas fontes.
        """,
        description="Agente que sugere possíveis fontes para a elaboração do quiz"
    )
    entrada = f"Quiz Original: {quiz_original}\nConteúdo Pesquisado: {conteudo_pesquisado}"
    fontes_sugeridas = call_agent(sugestor_fontes, entrada)
    return fontes_sugeridas

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def quiz_educacional():
    """Coleta dados do professor: disciplina, turma, conteúdo e inicia o fluxo."""

    titulo = widgets.HTML(
        value="""
        <h1 style='font-size: 2em; font-weight: bold; text-align:center;'>🎉 Bem-vindo(a) ao Prof+! 🤖</h1>
        <h2 style='font-size: 1.5em; text-align:center;'>✨ Seu assistente para criar quizzes incríveis! ✨</h2>
        <p style='font-size: 1.2em; text-align:center;'>📚 Vamos começar a elaborar as questões para seus alunos! 🚀</p>
        <p style='font-size: 1.2em; text-align:center;'>Por favor, preencha os campos abaixo com as informações necessárias:</p>
        """
    )


    disciplina_input = widgets.Text(description="📚 Disciplina:")
    disciplina_input.layout.width = '300px'

    turma_input = widgets.Text(description="🧑‍🏫 Turma (Ano):")
    turma_input.layout.width = '300px'

    conteudo_input = widgets.Text(description="📝 Conteúdo:")
    conteudo_input.layout.width = '300px'


    button = widgets.Button(description="💡 Gerar Quiz!", button_style='success')
    button.layout.width = '300px'

    output = widgets.Output()


    input_widgets = widgets.VBox([
        disciplina_input,
        turma_input,
        conteudo_input,
        button
    ])
    input_widgets.layout.align_items = 'center'

    def on_button_clicked(b):
        with output:
            output.clear_output()
            disciplina = disciplina_input.value
            turma = turma_input.value
            conteudo = conteudo_input.value

            if not disciplina or not turma or not conteudo:
                print("⚠️ Por favor, preencha todos os campos para prosseguir! ⚠️")
            else:
                try:
                    print("\n✅ Informações recebidas! Preparando o quiz... 🤓")
                    print(f"Disciplina: {disciplina}, Turma: {turma}, Conteúdo: {conteudo}\n")

                    print("#" * 120)
                    print("✨ Iniciando o processo de geração do quiz... ✨\n")
                    print("#" * 120)
                    informacoes = agente_buscador(conteudo)
                    print("\n--- 🔍 Resultado do Agente 1 (Buscador) ---\n")
                    print("--- ℹ️ Informações Encontradas ---")
                    print(informacoes)
                    print("#" * 120)

                    print("\n✨ Gerando as perguntas... ✍️\n")
                    print("#" * 120)
                    quiz = agente_gerador_quiz(informacoes, disciplina, turma)
                    print("\n--- 📝 Resultado do Agente 2 (Gerador) ---\n")
                    print("--- ❓ Quiz Gerado ---")
                    print(quiz)
                    print("#" * 120)

                    print("\n✨ Revisando e formatando as perguntas... 🧐\n")
                    print("#" * 120)
                    quiz_revisado_formatado = agente_revisor_formatador(quiz)
                    print("\n--- 📝 Resultado do Agente 3 (Revisor e Formatador) ---\n")
                    print("--- ✅ Quiz Revisado e Formatado ---")
                    print(quiz_revisado_formatado)
                    print("#" * 120)

                    print("\n✨ Solucionando o quiz... 💡\n")
                    print("#" * 120)
                    quiz_solucionado = agente_solucionador(quiz_revisado_formatado)
                    print("\n--- 📝 Resultado do Agente 4 (Solucionador) ---\n")
                    print("--- 🎯 Quiz Solucionado ---")
                    print(quiz_solucionado)
                    print("#" * 120)

                    print("\n✨ Gerando explicações detalhadas... 🧐\n")
                    print("#" * 120)
                    explicacoes = agente_gerador_explicacoes(quiz_revisado_formatado, quiz_solucionado, conteudo)
                    print("\n--- 📝 Resultado do Agente 5 (Gerador de Explicações Detalhadas) ---\n")
                    print("--- 💡 Explicações Detalhadas ---")
                    print(explicacoes)
                    print("#" * 120)

                    print("\n✨ Sugerindo as possíveis fontes... 📚\n")
                    print("#" * 120)
                    fontes = agente_indicador_fontes(quiz, conteudo)
                    print("\n--- 📚 Resultado do Agente 6 (Sugestão de Fontes) ---\n")
                    print("--- 🔍 Sugestões de Fontes ---")
                    print(fontes)
                    print("#" * 120)

                    print("\n🎉 Quiz completo! Pronto para usar! 🎉\n")

                except Exception as e:
                    print(f"🚨 Ocorreu um erro: {e} 🚨")

    button.on_click(on_button_clicked)


    interface = widgets.VBox([
        titulo,
        input_widgets,
        output
    ])
    interface.layout.align_items = 'center'

    display(interface)


In [ ]:
# Fluxo Principal
quiz_educacional()